In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split as tst

In [2]:
class logistic():
    def __init__(self,cl1,cl2):
        self.class1 = cl1
        self.class2 = cl2
        self.theta = None
        
    def pred(self,theta,x):
        hx = self.hypothesis(theta,x)
        if hx>=0.5:
            return self.class2
        else:
            return self.class1
        
    def hypothesis(self,theta,x):
        return 1/(1+np.exp(-np.dot(theta,x)))
    
    def gradient(self,x,y,theta):
        grad = np.zeros(x.shape[1])
        m = x.shape[0]
        for ix in range(m):
            hx = self.hypothesis(theta,x[ix])
            grad += (y[ix]-hx)*x[ix]
        return grad/m
    
    def gradientascent(self,x,y,lr=0.01):
        self.theta = np.zeros(x.shape[1])
        for ix in range(500):
            self.theta = self.theta + lr*self.gradient(x,y,self.theta)
        return self.theta
    
    def train(self,x_train,y_train,lr=0.01):
        self.theta = self.gradientascent(x_train,y_train,lr)

In [3]:
ds = pd.read_csv("fashion-mnist_test.csv")

In [4]:
ds = ds.values
print(ds.shape)
ds = ds[:10000,:]

(10000, 785)


In [5]:
x = np.ones(ds.shape)
y = np.zeros(ds.shape[0])
x[:,1:] = ds[:,1:]
y[:] = ds[:,0]
# print(np.unique(y))
# print(x,y)
x_train,x_test,y_train,y_test = tst(x,y,test_size=0.2)

In [6]:
print(np.unique(y))
print(np.unique(x[:,0]))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
[ 1.]


In [7]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(8000, 785) (8000,) (2000, 785) (2000,)


In [8]:
llist = []
for ix in range(10):
    for nx in range(ix+1,10):
        print(ix,nx,y_train)
        pp = logistic(ix,nx)
        XX,YY = [],[]
        for mx in range(y_train.shape[0]):
            if(y_train[mx]==ix):
                YY.append(0)
                XX.append(x_train[mx])
            elif(y_train[mx]==nx):
                YY.append(1)
                XX.append(x_train[mx])
        XX = np.array(XX)
        YY = np.array(YY)
        pp.train(XX[:,:],YY[:])
        llist.append(pp)
#     print(ix,nx)

    

0 1 [ 8.  8.  5. ...,  4.  4.  5.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


0 2 [ 8.  8.  5. ...,  4.  4.  5.]
0 3 [ 8.  8.  5. ...,  4.  4.  5.]
0 4 [ 8.  8.  5. ...,  4.  4.  5.]
0 5 [ 8.  8.  5. ...,  4.  4.  5.]
0 6 [ 8.  8.  5. ...,  4.  4.  5.]
0 7 [ 8.  8.  5. ...,  4.  4.  5.]
0 8 [ 8.  8.  5. ...,  4.  4.  5.]
0 9 [ 8.  8.  5. ...,  4.  4.  5.]
1 2 [ 8.  8.  5. ...,  4.  4.  5.]
1 3 [ 8.  8.  5. ...,  4.  4.  5.]
1 4 [ 8.  8.  5. ...,  4.  4.  5.]
1 5 [ 8.  8.  5. ...,  4.  4.  5.]
1 6 [ 8.  8.  5. ...,  4.  4.  5.]
1 7 [ 8.  8.  5. ...,  4.  4.  5.]
1 8 [ 8.  8.  5. ...,  4.  4.  5.]
1 9 [ 8.  8.  5. ...,  4.  4.  5.]
2 3 [ 8.  8.  5. ...,  4.  4.  5.]
2 4 [ 8.  8.  5. ...,  4.  4.  5.]
2 5 [ 8.  8.  5. ...,  4.  4.  5.]
2 6 [ 8.  8.  5. ...,  4.  4.  5.]
2 7 [ 8.  8.  5. ...,  4.  4.  5.]
2 8 [ 8.  8.  5. ...,  4.  4.  5.]
2 9 [ 8.  8.  5. ...,  4.  4.  5.]
3 4 [ 8.  8.  5. ...,  4.  4.  5.]
3 5 [ 8.  8.  5. ...,  4.  4.  5.]
3 6 [ 8.  8.  5. ...,  4.  4.  5.]
3 7 [ 8.  8.  5. ...,  4.  4.  5.]
3 8 [ 8.  8.  5. ...,  4.  4.  5.]
3 9 [ 8.  8.  5. ...

In [9]:
print(len(llist))

45


In [10]:
def prediction(x,llist):
    pl = np.zeros(len(llist))
    count=0
    for ix in llist:
        pl[count] = ix.pred(ix.theta,x)
        count+=1
    idx,val = np.unique(pl,return_counts=True)
    ridx = np.argmax(val)
#     print(ridx,idx[ridx])
    return idx[ridx]

In [11]:
def accuracy(x,y,llist):
    acc = 0
    for ix in range(x.shape[0]):
#         print(prediction(x[ix],llist),'\t',y[ix])
        if(prediction(x[ix],llist)==y[ix]):
            acc+=1
#             print(acc)
    return acc/x.shape[0]

In [12]:
accuracy(x_test,y_test,llist)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


0.803

# sklearn one versus one

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
 clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(x_train, y_train)

In [15]:
print(clf.score(x_test,y_test))

0.821


# KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=11)
neigh.fit(x_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')

In [17]:
neigh.score(x_test,y_test)

0.8105